In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
import re

In [2]:
lyrs_abba = ['','','','','','','']
for m in range(0,7):
    filename = 'a' + str(m) +'.txt'
    with open(filename,'r') as f:
        lyrs_abba[m] = f.read()
        f.close()

In [3]:
lyrs_fleet = ['','','','','','','']
for m in range(0,7):
    filename = 'f' + str(m) +'.txt'
    with open(filename,'r') as f:
        lyrs_fleet[m] = f.read()
        f.close()

In [4]:
def Trtext(text):
    _text = re.sub('\n', ' ', text)
    _text = re.sub(r'[!(,)]', ' ', _text)
    _text = re.sub(r"['][\w]+", '', _text)
    return _text

In [5]:
text_all = []

In [6]:
lines_abba = []
for line in lyrs_abba:
    lines_abba.append(line.splitlines())


In [7]:
lines_fleet = []
for line in lyrs_fleet:
    lines_fleet.append(line.splitlines())


In [8]:
pd.DataFrame([j for i in lines_fleet for j in i])

,0
0,If you wake up and don't want to smile
1,
2,If it takes just a little while
3,
4,Open your eyes and look at the day
...,...
446,"Oh, no, no you can't disguise"
447,"(You can't disguise, no you can't disguise)"
448,Tell me lies
449,Tell me sweet little lies


In [9]:
df_abba = pd.DataFrame([j for i in lines_abba for j in i])
df_abba['Artist'] = 'ABBA'
df_abba

,0,Artist
0,I don't wanna talk about things we've gone thr...,ABBA
1,,ABBA
2,"Though it's hurting me, now it's history",ABBA
3,,ABBA
4,I've played all my cards and that's what you'v...,ABBA
...,...,...
410,Having the time of your life,ABBA
411,"Ooh, see that girl",ABBA
412,Watch that scene,ABBA
413,Dig in the dancing queen,ABBA


In [10]:
df_fleet = pd.DataFrame([j for i in lines_fleet for j in i])
df_fleet['Artist'] = 'fleet'
df_fleet

,0,Artist
0,If you wake up and don't want to smile,fleet
1,,fleet
2,If it takes just a little while,fleet
3,,fleet
4,Open your eyes and look at the day,fleet
...,...,...
446,"Oh, no, no you can't disguise",fleet
447,"(You can't disguise, no you can't disguise)",fleet
448,Tell me lies,fleet
449,Tell me sweet little lies,fleet


In [11]:
df_all = pd.concat([df_abba,df_fleet],ignore_index=True)

In [12]:
df_all.columns = ['lyrics','artists']

In [13]:
df_all.columns

Index(['lyrics', 'artists'], dtype='object')

In [14]:
X = df_all.lyrics
y = df_all.artists

In [15]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(X,y,test_size=0.3)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')

In [17]:
vec_train = tfidf.fit_transform(xtrain)

In [18]:
vec_test = tfidf.transform(xtest)

In [19]:
from sklearn.ensemble import RandomForestClassifier

In [20]:
m = RandomForestClassifier(n_estimators=2900, max_depth=7)

In [21]:
m.fit(vec_train,ytrain)

RandomForestClassifier(max_depth=7, n_estimators=2900)

In [22]:
m.score(vec_train,ytrain)

0.7128712871287128

In [23]:
m.score(vec_test,ytest)

0.7076923076923077

In [24]:
from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier(n_estimators=4000, max_depth=6)
etc.fit(vec_train,ytrain)

ExtraTreesClassifier(max_depth=6, n_estimators=4000)

In [25]:
etc.score(vec_train, ytrain)

0.7178217821782178

In [26]:
etc.score(vec_test, ytest)

0.7

In [27]:
lyrs_test = []
filename = 'taf.txt'
with open(filename,'r') as f:
    lyrs_test = f.read()
    f.close()

In [28]:

lines_test = lyrs_test.splitlines()

In [29]:
y_unseen = ['ABBA','ABBA','ABBA','ABBA','ABBA','fleet','fleet','fleet','fleet','fleet']

In [30]:
test_df = pd.DataFrame(lines_test)
X_unseen = test_df.copy()
test_df['Artist'] = y_unseen

In [34]:
test_df.rename(columns={0:'lyrics'},inplace=True)
test_df

,lyrics,Artist
lyrics,You seem so far away though you are standing near,ABBA
1,"You made me feel alive, but something died I fear",ABBA
2,I really tried to make it out,ABBA
3,I wish I understood,ABBA
4,"What happened to our love, it used to be so good",ABBA
5,"Need your soft voice, that talked to me at night",fleet
6,"I don't want you to worry, baby",fleet
7,I know we can make everything alright,fleet
8,"Listen to my plea, baby, bring it to me",fleet
9,Because I need your love so bad,fleet


In [36]:
X_unseen = list(test_df['lyrics'])


In [37]:
X_unseen

['You seem so far away though you are standing near',
 'You made me feel alive, but something died I fear',
 'I really tried to make it out',
 'I wish I understood',
 'What happened to our love, it used to be so good',
 'Need your soft voice, that talked to me at night',
 "I don't want you to worry, baby",
 'I know we can make everything alright',
 'Listen to my plea, baby, bring it to me',
 'Because I need your love so bad']

In [38]:
vec_unseen = tfidf.transform(X_unseen)

In [39]:
vec_unseen

<10x300 sparse matrix of type '<class 'numpy.float64'>'
	with 19 stored elements in Compressed Sparse Row format>

In [40]:
vec_pred = m.predict(vec_unseen)

In [41]:
vec_pred

array(['fleet', 'ABBA', 'ABBA', 'fleet', 'fleet', 'ABBA', 'fleet', 'ABBA',
       'fleet', 'fleet'], dtype=object)

In [42]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB(alpha=1)
mnb.fit(vec_train,ytrain)
mnb.score(vec_train,ytrain)

0.806930693069307

In [43]:
vec_pred = mnb.predict(vec_unseen)
vec_pred

array(['ABBA', 'ABBA', 'ABBA', 'fleet', 'fleet', 'ABBA', 'fleet', 'ABBA',
       'fleet', 'fleet'], dtype='<U5')